In [1]:
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time

In [2]:
import pandas as pd

In [3]:
#chromedriver_path = "C:\Program Files (x86)\chromedriver.exe"

In [4]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [5]:
# technology jobs
url1 = 'https://careers.microsoft.com/us/en/search-results?keywords=technology&from='
url2 = '&s=1'

In [7]:
def microsoft_get_jobs(url1, url2):
    """
    retrieve job title and job link from every page in the URL
    number_page -- Number of pages that one wish to retrive the data from.
    Return: A tuple including all the job infromation for each job in each page
    """
    job_title=[]
    job_link=[]
    page_num = 0

    driver=webdriver.Chrome(options = chrome_options)
    URL = url1 + str((page_num-1)*20) + url2
    driver.get(URL)
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    while soup.select("span[class=job-title]") :
        job_title.extend([td.text for td in soup.select("span[class=job-title]")])
        job_link.extend([td.find('a').get('href') for td in soup.findAll("div", {"class": "information"})])
        driver.quit()

        page_num += 1
        driver=webdriver.Chrome(options = chrome_options)
        URL = url1 + str((page_num-1)*20) + url2
        driver.get(URL)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    driver.quit() 
    return job_title,job_link

In [9]:
job_title, job_link = microsoft_get_jobs(url1, url2)

In [11]:
job_title[0:5]

['Lead Technical Animator – Gears of War – The Coalition',
 'Software Engineer - CTJ',
 'ESO Datacenter Operations Specialist',
 'ESO Datacenter Program Manager',
 'Senior Software Engineer']

In [33]:
def remove_space(s):
  s_new = s.replace('\xa0', '').replace('\n', '')
  while '  ' in s_new:
    s_new = s_new.replace('  ', ' ')
  return s_new

In [12]:
# create a dataframe that contains job titles and links for all job categories
df_title_link = pd.DataFrame(zip(job_title, job_link), columns=['JOB_TITLE', 'JOB_LINK'])

In [13]:
df_title_link.head()

,JOB_TITLE,JOB_LINK
0,Lead Technical Animator – Gears of War – The C...,https://careers.microsoft.com/us/en/job/150444...
1,Software Engineer - CTJ,https://careers.microsoft.com/us/en/job/151261...
2,ESO Datacenter Operations Specialist,https://careers.microsoft.com/us/en/job/151199...
3,ESO Datacenter Program Manager,https://careers.microsoft.com/us/en/job/151198...
4,Senior Software Engineer,https://careers.microsoft.com/us/en/job/148279...


In [14]:
# drop the duplicates
df_title_link = df_title_link.drop_duplicates()
print(len(df_title_link))

1154


In [15]:
# save the titles and links to csv
df_title_link.to_csv('C:\Online_Courses\GitHub\Swatcloud\data\web_scraping\data\microsoft_jobs_titles_links_all.csv')

## Extract job descriptions and qualifications for each job

In [16]:
# load in the job titles and links from the csv file
df_title_link = pd.read_csv('C:\Online_Courses\GitHub\Swatcloud\data\web_scraping\data\microsoft_jobs_titles_links_all.csv', index_col=0)
df_title_link.head()

,JOB_TITLE,JOB_LINK
0,Lead Technical Animator – Gears of War – The C...,https://careers.microsoft.com/us/en/job/150444...
1,Software Engineer - CTJ,https://careers.microsoft.com/us/en/job/151261...
2,ESO Datacenter Operations Specialist,https://careers.microsoft.com/us/en/job/151199...
3,ESO Datacenter Program Manager,https://careers.microsoft.com/us/en/job/151198...
4,Senior Software Engineer,https://careers.microsoft.com/us/en/job/148279...


In [27]:
# retrieve job qualifications and descriptions
def job_description(title, link):
    qualifications = []
    description = []
    jobtitle = []
    joblink = []

    driver=webdriver.Chrome('chromedriver',options=chrome_options)
    for i in range(len(link)):
        URL=link[i]
        driver.get(URL)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        s = ''
        d = ''

        try:
          tag = soup.find('p', {'data-ph-at-id' : 'job-qualifications-text'}).find_all('li')
          for t in tag:
            s = s + '. ' + t.text
        except:
          print(i, 'no qual')
        
        try:
          tag = soup.find('p', {'data-ph-at-id' : 'job-responsibilities-text'})
          d =  tag.text
        except:
          print(i, 'no qual')
        
        if len(s) > 10:
            qualifications.append(s)
            description.append(d)   
            jobtitle.append(title[i])
            joblink.append(link[i])
        
    driver.quit()            
 
    return jobtitle, link, qualifications, description

In [31]:
# retrieve the qualification for each job.
title, link, qual, descrp = job_description(df_title_link['JOB_TITLE'].values, df_title_link['JOB_LINK'].values)

1023 no qual


In [38]:
qual_cleaned = [remove_space(q) for q in qual]
descrp_cleaned = [remove_space(d) for d in descrp]

In [44]:
#create a dataframe from basic qualification and preferred qualification
df = pd.DataFrame(zip(title, qual_cleaned, link, descrp_cleaned))
df.columns = ['TITLE', 'QUALIFICATIONS', 'LINK', 'DESCRIPTION']
df['COMPANY'] = 'Microsoft'
df = df.iloc[:, [4, 0, 1, 2, 3]]
df.head()

,COMPANY,TITLE,QUALIFICATIONS,LINK,DESCRIPTION
0,Microsoft,Lead Technical Animator – Gears of War – The C...,Must have 5+ years of experience in AAA video...,https://careers.microsoft.com/us/en/job/150444...,"Collaborate with animation leads, and engineer..."
1,Microsoft,Software Engineer - CTJ,"Bachelor's Degree in Computer Science, or rel...",https://careers.microsoft.com/us/en/job/151261...,Works with appropriate stakeholders to determi...
2,Microsoft,ESO Datacenter Operations Specialist,High School Diploma or equivalent AND 1+ year...,https://careers.microsoft.com/us/en/job/151199...,"As a Datacenter Operations Specialist, your re..."
3,Microsoft,ESO Datacenter Program Manager,High School Diploma or equivalent AND 1+ year...,https://careers.microsoft.com/us/en/job/151198...,"As a Edge Site Operations DCPM,your responsibi..."
4,Microsoft,Senior Software Engineer,4+ years of experience in software developmen...,https://careers.microsoft.com/us/en/job/148279...,Our solution is built using a mix of web techn...


In [45]:
# look for empty QUALIFICATION entries
def get_empty(df, colname) :
  empty_idx=[]
  for i in range(len(df[colname])):
    count = len(df[colname][i])
    if count<=10 : empty_idx.append(i)
    else: pass
  return empty_idx

empty_mini_q = get_empty(df, 'QUALIFICATIONS')
print(len(empty_mini_q))

# drop the empty Qualification entries
df.drop(empty_mini_q, inplace=True)
df.shape

0


(1056, 5)

In [46]:

df_nodup.shape

(1056, 5)

In [47]:
#remove the duplicated jobs
df_nodup = df.drop_duplicates()
print("There are {} jobs from Microsoft.".format(df_nodup.shape[0]))

df_nodup.to_csv('microsoft_technology_jobs_cleaned.csv')